In [1]:
! git clone https://github.com/kaspergroenbek98/first-first-year-project.git

fatal: destination path 'first-first-year-project' already exists and is not an empty directory.


In [2]:
import numpy as np
import nltk
import csv
import nltk
nltk.download('punkt')
nltk.download('stopwords')
stopwords = set(nltk.corpus.stopwords.words('english'))


# Read the data into a header and a data np.array - the array is then shuffled
with open('first-first-year-project/data.csv', encoding='latin1') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    data = np.array([np.array(line) for line in csv_reader])
header, data = data[0,:], data[1:,:]
print(header)
np.random.shuffle(data)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['other_topic' 'resolution_topics' 'gender' 'name' 'Resolution_Category'
 'retweet_count' 'text' 'tweet_coord' 'tweet_created' 'tweet_date'
 'tweet_id' 'tweet_location' 'tweet_state' 'user_timezone' 'tweet_region']


In [0]:
porter = nltk.PorterStemmer()

# Remove all stopwords, non-alphabet words (except spaces), and stem the words
for i, row in enumerate(data[:,6]):
    row = row.lower()
    row = ''.join(char for char in row if char.isalpha() or char == ' ')
    row = ' '.join(porter.stem(word) for word in row.split() if word not in stopwords)
    data[i,6] = row

# Create a long string of ALL words, then token them, and create a frequency distribution
actualWords = ' '.join(token for sentence in data[:,6] for token in sentence.split())
#tokens = nltk.word_tokenize(actualWords)
tokens = actualWords.split()
fd = nltk.FreqDist(tokens)


In [0]:
word_features = [word for (word, count) in fd.most_common(2000)]

# Compares the words of a specfic tweet with the top 2000 most-used words
def tweet_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features['contains(%s)' % word] = (word in tweet_words)
    return features

""" 
Creates tuples with a vector containing boolean values depending on whether
or not the word is in the tweet - along with the label of the tweet.
"""
size = 1000
featuresets = [(tweet_features(d.split()), c) for (d,c) in zip(data[:size,6], data[:size,2])] # column 6 is text data, column 2 is gender data
train_set, test_set = featuresets[size//2:], featuresets[:size//2]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [5]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.554
Most Informative Features
           contains(fit) = True           female : male   =      5.0 : 1.0
contains(hashtagoftheweek) = True           female : male   =      5.0 : 1.0
      contains(cariloha) = True           female : male   =      4.2 : 1.0
           contains(big) = True             male : female =      3.8 : 1.0
        contains(finish) = True           female : male   =      3.4 : 1.0
          contains(girl) = True             male : female =      3.2 : 1.0
           contains(amp) = True           female : male   =      3.0 : 1.0
         contains(everi) = True             male : female =      3.0 : 1.0
          contains(goal) = True           female : male   =      2.8 : 1.0
          contains(self) = True           female : male   =      2.7 : 1.0


In [0]:
bigrm = list(nltk.ngrams(tokens, 2))
fdbigrm = nltk.FreqDist(bigrm)


In [0]:
word_features = list(fdbigrm.keys())[:2000]

def tweet_features(tweet):
    tweet_words = set(tweet)
    features = {}
    for word in word_features:
        features[f'contains({word})'] = (word in tweet_words)
    return features
size = 1000
featuresets = [(tweet_features(nltk.ngrams(d.split(), 2)), c) for (d,c) in zip(data[:size,6], data[:size,2])] # column 6 is text data, column 2 is gender data
train_set, test_set = featuresets[size//2:], featuresets[:size//2]
classifier = nltk.NaiveBayesClassifier.train(train_set)

In [8]:
print(nltk.classify.accuracy(classifier, test_set))
classifier.show_most_informative_features(10)

0.5
Most Informative Features
contains(('make', 'money')) = True           female : male   =      2.7 : 1.0
contains(('newyearsresolut', 'lose')) = True           female : male   =      2.7 : 1.0
contains(('get', 'back')) = True             male : female =      2.0 : 1.0
contains(('year', 'newyearsresolut')) = True             male : female =      1.9 : 1.0
contains(('resolut', 'go')) = True           female : male   =      1.9 : 1.0
contains(('chrisbrown', 'supper')) = True           female : male   =      1.9 : 1.0
contains(('smoke', 'cigarett')) = True           female : male   =      1.9 : 1.0
contains(('think', 'newyearsresolut')) = True           female : male   =      1.9 : 1.0
 contains(('go', 'tri')) = True           female : male   =      1.9 : 1.0
contains(('money', 'thank')) = True           female : male   =      1.9 : 1.0


In [9]:
'newyearsresolution' in stopwords

False